<a href="https://colab.research.google.com/github/mk-a/NLP_Duty1/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import pickle
import torch.nn as nn
import time

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/IFT6285/train_post_preprocessed.pkl")

In [0]:
a = np.array([len(post) for post in data.text.values if len(post)<1000 ])

In [0]:
def text2seq(X, y, vocab_max, seq_len, unk_tok='UNK', padd_tok='PAD' ):
  """ For a list of inputs X with different sequence length, associated to the labels y.
      Builds the list of the vocab_max-2 most common words. Maps each of them to a number.
      Replace the other words by the token unk_tok, which is mapped to the value 1.
      Then force the input to have a sequence length of seq_len with this policy:
        - If the sample x is short than seq_len, then adds padding at the begining.
          The padding token is set by the variable padd_tok. And mapped to the value 0.
        - If the sample x is longer than seq_len, then extracts (len(x)//seq_len)+1
          sequences from x.
  """
  count_words = Counter()
  for post in X:
    for tok in post:
      count_words[tok] += 1
  vocab = {w for w,_ in count_words.most_common(vocab_max-2)}.union( {unk_tok, padd_tok})
  word2val = {w : i+2 for i, (w,_) in enumerate( count_words.most_common(vocab_max-2) )}
  word2val[padd_tok] = 0
  word2val[unk_tok] = 1

  X2 = []
  y2 = []
  for i, x in enumerate(X):
    if len(x) < seq_len:
      tmp = x.copy()
      for _ in range(seq_len - len(x)):
        tmp.insert(0, padd_tok)
      X2.append([word2val[tok] if tok in vocab else 1 for tok in tmp])
      y2.append(y[i])
    elif len(x) == seq_len:
      X2.append([word2val[tok] if tok in vocab else 1 for tok in x]) 
      y2.append(y[i])
    else :
      for j in range(len(x)//seq_len):
        X2.append([word2val[tok] if tok in vocab else 1 for tok in  x[j*seq_len:(j+1)*seq_len]])
        y2.append(y[i])
      X2.append([word2val[tok] if tok in vocab else 1 for tok in  x[-seq_len:]])
      y2.append(y[i])
  return word2val, X2, y2

In [0]:
vocab_max = 80000
word2val, X, y = text2seq(data.text.values, data.label.values, vocab_max,  200)

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/IFT6285/word2val_80k.pkl', 'wb') as handle:
    pickle.dump(word2val, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/My Drive/Colab Notebooks/IFT6285/X_v80k_s200.pkl', 'wb') as handle:
    pickle.dump(X, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/My Drive/Colab Notebooks/IFT6285/y_s200.pkl', 'wb') as handle:
    pickle.dump(y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/IFT6285/word2val_80k.pkl', 'rb') as handle:
    word2val = pickle.load(handle)
with open('/content/drive/My Drive/Colab Notebooks/IFT6285/X_v80k_s200.pkl', 'rb') as handle:
    X = pickle.load(handle)
with open('/content/drive/My Drive/Colab Notebooks/IFT6285/y_s200.pkl', 'rb') as handle:
    y = pickle.load(handle)

In [0]:
def downsample(df, colum_name):
    size_min = float('inf')
    list_df = []
    labels = df[colum_name].unique()
    for label in labels:
        size = len(df.loc[df[colum_name] == label])
        if size < size_min:
            size_min = size
    for label in labels:
        list_df.append(df.loc[df[colum_name] == label].sample(size_min))
    return pd.concat(list_df).sample(frac=1).reset_index(drop=True)

In [0]:
df = downsample(pd.DataFrame({'X':X, 'y':y}), 'y')

In [0]:
X2 = list(df.X.values)
y2 = list(df.y.values)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X2, y2, test_size=0.33, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=42)
X_train = torch.tensor(X_train)
X_valid = torch.tensor(X_valid)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_valid = torch.tensor(y_valid)
y_test = torch.tensor(y_test)
train_data = torch.utils.data.TensorDataset(X_train, y_train)
valid_data = torch.utils.data.TensorDataset(X_valid, y_valid)
test_data = torch.utils.data.TensorDataset(X_test, y_test)

In [0]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [0]:
class RNN(nn.Module):
    def __init__(self,
                 emb_size,
                 hidden_size,
                 vocab_size,
                 num_layers,
                 output_size,
                 nonlinearity = 'relu',
                 bias = True,
                 dropout = 0,
                 bidirectional = True
                ):
        super(RNN, self).__init__()
        #hyper-parameters
        self.emb_size      = emb_size
        self.hidden_size   = hidden_size
        self.vocab_size    = vocab_size
        self.num_layers    = num_layers
        self.output_size   = output_size
        self.nonlinearity  = nonlinearity
        self.bias          = bias
        self.dropout       = dropout
        self.bidirectional = bidirectional
        #layers
        self.embedding = nn.Embedding(vocab_size, emb_size).to(device)
        self.rnn = nn.RNN(input_size = emb_size,
                          hidden_size = hidden_size,
                          num_layers = num_layers,
                          bias = bias,
                          dropout = dropout,
                          bidirectional = bidirectional
                         ).to(device)
        self.linear = nn.Linear((bidirectional+1) *hidden_size, output_size).to(device)
        
    def forward(self, X):
        out = self.embedding(X.t())
        out, _ = self.rnn(out)
        return self.linear(out[-1])

In [0]:
def compute_accuracy(model, data_loader):
  correct = 0
  for i, (X_batch, y_batch) in enumerate(data_loader):
    torch.cuda.empty_cache()
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)
    out = model.forward(X_batch.to(device))
    correct += (torch.max(out, 1)[1] == y_batch).float().sum()
  return((100*correct/len(test_loader.dataset)).item())

In [0]:
def run_epoch(n_epochs, model, criterion, optimizer, train_loader, valid_loader=None):
  start_time = time.time()
  last_time = time.time()
  for epoch in range(1, n_epochs+1):
      print("Epoch: {}/{}".format(epoch, n_epochs))
      running_loss = 0
      correct = 0
      for i, (X_batch, y_batch) in enumerate(train_loader):
          torch.cuda.empty_cache()
          X_batch = X_batch.to(device)
          y_batch = y_batch.to(device)
          optimizer.zero_grad()
          out = model.forward(X_batch.to(device))
          loss = criterion(out, y_batch)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
          correct += (torch.max(out, 1)[1] == y_batch).float().sum()
          if time.time() - last_time > 0.5:
              print("Samples:{}/{}\tloss: {:.4f}\tacc: {:2.3f}\telapsed_time: {:.1f}s"\
                    .format( (i+1)*batch_size, len(train_loader.dataset),\
                            running_loss/((i+1)*batch_size), 100*correct/((i+1)*batch_size), time.time()-start_time), end='\r')
              last_time = time.time()
      if valid_loader:
        print("loss: {:.6f}\tacc: {:2.3f}\telapsed_time: {:.1f}s\tvalid_acc: {:2.3f}".format(
                running_loss/((i+1)*batch_size), 100*correct/((i+1)*batch_size), time.time()-start_time,
                compute_accuracy(model,valid_loader)
        ))
      else:
        print("loss: {:.6f}\tacc: {:2.3f}\telapsed_time: {:.1f}s".format(
                running_loss/((i+1)*batch_size), 100*correct/((i+1)*batch_size), time.time()-start_time,
        ))


In [0]:
rnn = RNN( emb_size = 256,
         hidden_size = 512,
         vocab_size = vocab_max,
         num_layers = 2,
         output_size = 3,
         bidirectional=False)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters())

In [0]:
run_epoch(n_epochs=10, model=rnn, criterion=criterion, optimizer=optimizer, train_loader=test_loader, valid_loader=valid_loader)

Epoch: 1/10
